# Enchance Anki cards with OpenAI API

This notebook demonstrates how to enhance Anki cards with OpenAI API. In thix example the word-translation cards are enchanced with transcription, meaning, synonyms, and examples. 

In [1]:
# !pip -r requirements.txt

In [2]:
import csv
import pandas as pd

from io import StringIO
from openai import OpenAI
from tqdm.auto import tqdm
from utils import parse_anki_xml, save_anki_xml, SETTINGS_NAMES_LIST, SYSTEMPROMPT, USERPROMPT

/home/arabella/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Initialize OpenAI API with your key
API_KEY = ...
# Number of cards operated per prompt. The more cards, the less times
# you need to send the request(=lowe imput tokens consumption).
# But the more cards, the less precise they are.
CARDS_PER_PROMPT = 10
INPUT_FILE = "example.zip"
OUTPUT_FILE = "example_enhanced.zip"

Some settings for AnkiApp iOS file format.

In [4]:
# available settings:
# @fontSizePx - font size in pixels
# @sides - '01' for back-sided, '11' for two-sided, '10' for front-sided
# @fieldType - 'text' for text fields, other not supported
SETTINGS_NAMES_LIST = ["@fontSizePx", "@sides", "@fieldType"]

# DataFrame with ouput field specifications. Columns are field names, rows are field settings.
OUTPUT_SETTINGS = pd.DataFrame(
    data={
        "word": ["22", "11", "text"],
        "transcription": ["22", "01", "text"],
        "meaning": ["15", "01", "text"],
        "synonyms": ["15", "01", "text"],
        "translation": ["20", "01", "text"],
        "example": ["12", "01", "text"],
    },
    index=SETTINGS_NAMES_LIST,
)
OUTPUT_SETTINGS

,word,transcription,meaning,synonyms,translation,example
@fontSizePx,22,22,15,15,20,12
@sides,11,01,01,01,01,01
@fieldType,text,text,text,text,text,text


### Read and parse the input deck

In this example, I use a dummy deck with 5 cards from AnkiApp is used. I prefer this format, because it allows you to specify fontsized and so on. For other apps, export the deck in `.csv` format and transform it to similar structure.

In [17]:
# Example deck with 5 messy cards
input_data, input_settings = parse_anki_xml(INPUT_FILE)
input_data

,Back,Front
id,,
0,очаровательный,fascinating
1,surge,всплеск
2,насморк,the sniffles
3,"ученик, подмастерье",apprentice
4,venture to,осмелиться


In [6]:
input_settings

,Back,Front
@fontSizePx,22,22
@sides,01,11
@fieldType,text,text


### Query ChatGPT

I leverage ChatGPT to enhance my anki deck: I will ask it to transform dummy `word`-`translation` card pairs into cards with `word`, `translation`, `transcription`, `meaning`, `synonyms`, and `examples`. If your deck has another structure, modify the `userprompt` accordingly.

In [7]:
# this is how request for the first card batch will look like
print(
    USERPROMPT.format(
        input_data.iloc[:CARDS_PER_PROMPT].to_csv(
            index=False, quoting=csv.QUOTE_ALL, quotechar='"'
        )
    )
)


I use this deck for learning English.
I fill it manually and a bit messy, and I want you to make them more accurate.
My deck has the following columns: Front(=word), and Back(=translation).

I want you to 
1. Normalize, remove prepositions, and fix missprints in the 'word' field. 
2. Look up the meaning, transcription, synonyms (separated by "|"), and 2-3 usage B2-level examples (separated by "\n"). If many options are available, select one that aligns well with my 'translation' field. All these fields are in English.
3. The 'word' can be in English or in Russian. If the 'word; is in English, 'translation' would be Russian and vice versa.

I have an example for you:

Input: 
```csv
word,translation
fledgling,новичок
снисходительный,condescending
```

Output:
```csv
word,translation,transcription,meaning,synonyms,example
"снисходительный","condescending","/ˌkɒndɪˈsɛndɪŋ/","having or showing an attitude of patronizing superiority","patronizing | disdainful | haughty","Her condescending 

In [8]:
client = OpenAI(api_key=API_KEY)

In [16]:
data = [] # enchanced cards go here
# To store invalid responses. ChatGPT is not guaranteed to return
# a valid CSV, althought I got 0 invalid respoonses of 500
bad_responses = []
log = []  # to store all the requests and responses (debug needs)

for i in tqdm(range(0, len(input_data), CARDS_PER_PROMPT)):
    messages = [
        {"role": "system", "content": SYSTEMPROMPT },
        {
            "role": "user",
            "content": USERPROMPT.format(
                input_data.iloc[i : i + CARDS_PER_PROMPT].to_csv(
                    index=False, quoting=csv.QUOTE_ALL, quotechar='"'
                )
            ),
        },
    ]
    stream = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=messages,
        stream=True,
    )
    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            cont = chunk.choices[0].delta.content
            response += cont
    log.append((messages, response))
    response = response.replace("```csv\n", "")
    response = response.replace("```", "")
    csvStringIO = StringIO(response)
    try:
        data.append(pd.read_csv(csvStringIO, sep=","))
    except:
        bad_responses.append(response)
        print(f"[Error] Bad response {i}")
        continue
enchanced_cards = pd.concat(data)

  0%|                                                     | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [15]:
# save & preview of the enhanced cards
enchanced_cards.to_csv("csv/enchanced_cards.csv", index=False)
OUTPUT_SETTINGS.to_csv("csv/enchanced_settings.csv")
enchanced_cards.head()

,word,translation,transcription,meaning,synonyms,example
0,fascinating,очаровательный,/ˈfæsɪneɪtɪŋ/,extremely interesting or charming,captivating | enchanting | intriguing,The documentary was absolutely fascinating and...
1,surge,всплеск,/sɜːrdʒ/,"a sudden powerful forward or upward movement, ...",rush | wave | increase,There was a surge in demand for face masks dur...
2,the sniffles,насморк,/ðə ˈsnɪflz/,a slight illness in which someone has a runny ...,cold | rhinitis | sneezing,I caught the sniffles after walking in the rai...
3,apprentice,"ученик, подмастерье",/əˈprɛntɪs/,a person who is learning a trade from a skille...,trainee | learner | novice,"As an apprentice, he learned the art of carpen..."
4,venture,осмелиться,/ˈvɛntʃər/,to dare to do something or to go somewhere tha...,dare | risk | attempt,He decided to venture into the wilderness desp...


### Saving resulting deck

Now you can save the enchanced deck to a file (it is located in `decks`) folder. 
The next steps are:
1. Send the file to your device (via messengers, email, etc.)
2. On device, click "Save in Files" iOS button
3. Then open AnkiApp, click "+" -> "Import Deck"-> "AnkiApp" -> "Choose file" -> Select your file
4. For other apps, you can use `.csv` format deck - it can be found in `csv` folder.
5. Done!

In [12]:
save_anki_xml(OUTPUT_FILE, "LLM enchanced deck", enchanced_cards, OUTPUT_SETTINGS)